In [1]:
# !rm -rf tmp_out

In [2]:
!pip install torcheeg torch-scatter torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.4/251.4 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.5/231.5 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of mne to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of wfdb to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from torcheeg.datasets import SEEDIVDataset
from torcheeg import transforms
import scipy.signal as signal
import random
import copy
from torch import Tensor
from torchvision.models.googlenet import BasicConv2d
from typing import Callable, Optional
import numpy as np

In [4]:
batchSize = 32
dropout = 0.5
lr = 1e-4
weightDecay = 1e-4
epochsCount = 100

In [5]:
# 1. Setup Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
def BandPassFilter(eeg_data):
    b, a = signal.butter(4, Wn=[1.0, 75.0], btype='bandpass', fs=200)
    return signal.filtfilt(b, a, eeg_data, axis=-1)

In [7]:
def Notch(eeg_data):
    b, a = signal.iirnotch(w0=50.0, Q=30.0, fs=200)
    return signal.filtfilt(b, a, eeg_data, axis=-1)

In [8]:
# 2. Define Preprocessing
t_transform = transforms.Compose([
    transforms.Lambda(BandPassFilter),
    transforms.Lambda(Notch),
    transforms.BaselineRemoval(),
    transforms.MeanStdNormalize(),
    transforms.To2d()
])

In [9]:
# 3. Load Data
dataset = SEEDIVDataset(
    io_path='./tmp_out/seed_iv',
    root_path='/kaggle/input/seed-iv/eeg_raw_data',
    offline_transform=t_transform,
    label_transform=transforms.Compose([
        transforms.Select('emotion'),
    ]),
    chunk_size=800,  # 4 seconds
    num_worker=4
)

[2025-12-06 14:54:47] INFO (torcheeg/MainThread) 🔍 | Processing EEG data. Processed EEG data has been cached to ./tmp_out/seed_iv.
[2025-12-06 14:54:47] INFO (torcheeg/MainThread) ⏳ | Monitoring the detailed processing of a record for debugging. The processing of other records will only be reported in percentage to keep it clean.
[PROCESS]:   0%|          | 0/45 [00:00<?, ?it/s]
[RECORD /kaggle/input/seed-iv/eeg_raw_data/1/4_20151111.mat]: 0it [00:00, ?it/s]
[RECORD /kaggle/input/seed-iv/eeg_raw_data/1/4_20151111.mat]: 1it [00:03,  3.93s/it]
[RECORD /kaggle/input/seed-iv/eeg_raw_data/1/4_20151111.mat]: 7it [00:04,  2.34it/s]
[RECORD /kaggle/input/seed-iv/eeg_raw_data/1/4_20151111.mat]: 13it [00:04,  5.06it/s]
[RECORD /kaggle/input/seed-iv/eeg_raw_data/1/4_20151111.mat]: 20it [00:04,  9.11it/s]
[RECORD /kaggle/input/seed-iv/eeg_raw_data/1/4_20151111.mat]: 27it [00:04, 14.01it/s]
[RECORD /kaggle/input/seed-iv/eeg_raw_data/1/4_20151111.mat]: 33it [00:04, 18.73it/s]
[RECORD /kaggle/input/s

In [10]:
# 1. Get the metadata DataFrame
df = dataset.info

# 2. Count the segments for each emotion
# 0: Neutral, 1: Sad, 2: Fear, 3: Happy
counts = df['emotion'].value_counts().sort_index()
total = len(df)

print(f"Total Segments: {total}")
print("-" * 30)
print("Count per Emotion:")
print(counts)

print("-" * 30)
print("Percentage per Emotion:")
percentages = (counts / total) * 100
print(percentages.round(2))

# 3. Check for Imbalance
# If the difference between max and min is > 10%, we might need a WeightedSampler
max_pct = percentages.max()
min_pct = percentages.min()

if (max_pct - min_pct) > 10:
    print(f"\n⚠️ WARNING: Data is IMBALANCED (Diff: {max_pct - min_pct:.2f}%)")
    print("Consider using a WeightedRandomSampler.")
else:
    print(f"\n✅ Data is reasonably BALANCED (Diff: {max_pct - min_pct:.2f}%)")

Total Segments: 37575
------------------------------
Count per Emotion:
emotion
0    10170
1    10245
2     9225
3     7935
Name: count, dtype: int64
------------------------------
Percentage per Emotion:
emotion
0    27.07
1    27.27
2    24.55
3    21.12
Name: count, dtype: float64

✅ Data is reasonably BALANCED (Diff: 6.15%)


In [11]:
# Split by Trial ID
# SEED-IV has 24 trials (videos) per session.
# 80% of VIDEOS for training (19 videos), 20% for testing (5 videos).
all_trial_ids = list(range(1, 25))

random.seed(42)
test_trial_ids = random.sample(all_trial_ids, 5)
train_trial_ids = [t for t in all_trial_ids if t not in test_trial_ids]

train_indices = df[df['trial_id'].isin(train_trial_ids)].index.tolist()
test_indices = df[df['trial_id'].isin(test_trial_ids)].index.tolist()

# Create Subsets & Loaders
train_set = Subset(dataset, train_indices)
test_set = Subset(dataset, test_indices)

train_loader = DataLoader(train_set, batch_size=batchSize, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batchSize, shuffle=False)

### Inception Class

In [12]:
class Inception(nn.Module):
    def __init__(
        self,
        in_channels: int,
        ch1x1: int,
        ch3x3red: int,
        ch3x3: int,
        ch5x5red: int,
        ch5x5: int,
        pool_proj: int,
        conv_block: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        if conv_block is None:
            conv_block = BasicConv2d
        self.branch1 = conv_block(in_channels, ch1x1, kernel_size=1)

        self.branch2 = nn.Sequential(
            conv_block(in_channels, ch3x3red, kernel_size=1),
            conv_block(ch3x3red, ch3x3, kernel_size=3, padding=1),
        )

        self.branch3 = nn.Sequential(
            conv_block(in_channels, ch5x5red, kernel_size=1),
            # Here, kernel_size=3 instead of kernel_size=5 is a known bug.
            # Please see https://github.com/pytorch/vision/issues/906 for details.
            conv_block(ch5x5red, ch5x5, kernel_size=3, padding=1),
        )

        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1, ceil_mode=True),
            conv_block(in_channels, pool_proj, kernel_size=1),
        )

    def _forward(self, x: Tensor) -> list[Tensor]:
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)

        outputs = [branch1, branch2, branch3, branch4]
        return outputs

    def forward(self, x: Tensor) -> Tensor:
        outputs = self._forward(x)
        return torch.cat(outputs, 1)

### GoogleNet class

In [13]:
printEnabled = True


def cprint(x):
    global printEnabled
    if printEnabled:
        print(x)


class GoogLeNetLighter(nn.Module):
    __constants__ = ["transform_input"]

    def __init__(
        self,
        num_classes: int = 4,
        transform_input: bool = False,
        init_weights: Optional[bool] = None,
        blocks: Optional[list[Callable[..., nn.Module]]] = None,
        dropout: float = 0.4,  # Increased dropout slightly
    ) -> None:
        super().__init__()
        if blocks is None:
            conv_block = BasicConv2d
            inception_block = Inception
        else:
            if len(blocks) != 2:
                raise ValueError(f"blocks length should be 2 instead of {len(blocks)}")
            conv_block = blocks[0]
            inception_block = blocks[1]

        if init_weights is None:
            init_weights = True

        self.transform_input = transform_input

        # 1. INPUT CHANNEL: 1 instead of 3 (62x800 input)
        self.conv1 = conv_block(1, 16, kernel_size=5, stride=1, padding=2)
        self.maxpool1 = nn.MaxPool2d((3, 5), stride=1, ceil_mode=True)

        self.conv2 = conv_block(16, 16, kernel_size=1)
        self.conv3 = conv_block(16, 48, kernel_size=3, padding=1)

        self.maxpool2 = nn.MaxPool2d(3, stride=2, ceil_mode=True)

        self.inception3a = inception_block(48, 32, 48, 64, 8, 16, 16)
        self.inception3b = inception_block(128, 64, 64, 96, 16, 48, 32)

        self.maxpool3 = nn.MaxPool2d(3, stride=2, ceil_mode=True)

        self.inception4a = inception_block(240, 96, 48, 104, 8, 24, 32)
        self.inception4b = inception_block(256, 80, 56, 112, 12, 32, 32)
        self.inception4c = inception_block(256, 64, 64, 128, 12, 32, 32)
        self.inception4d = inception_block(256, 56, 72, 144, 16, 32, 32)
        self.inception4e = inception_block(264, 128, 80, 160, 16, 64, 64)

        self.maxpool4 = nn.MaxPool2d((2, 4), stride=(2, 2), ceil_mode=True)

        self.inception5a = inception_block(416, 128, 80, 160, 16, 64, 64)
        self.inception5b = inception_block(416, 192, 96, 192, 24, 64, 64)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(p=dropout)

        self.fc = nn.Linear(512, num_classes)

        if init_weights:
            for m in self.modules():
                if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                    torch.nn.init.trunc_normal_(m.weight, mean=0.0, std=0.01, a=-2, b=2)
                elif isinstance(m, nn.BatchNorm2d):
                    nn.init.constant_(m.weight, 1)
                    nn.init.constant_(m.bias, 0)

    def _forward(self, x: Tensor) -> tuple[Tensor, Optional[Tensor], Optional[Tensor]]:
        cprint("*" * 30)
        # N x 1 x 62 x 800
        x = self.conv1(x)  # N x 32 x 62 x 800
        cprint(f"conv1       = {x.shape}")
        x = self.maxpool1(x)  # N x 32 x 62 x 400
        cprint(f"maxpool1    = {x.shape}")
        x = self.conv2(x)  # N x 32 x 62 x 400
        cprint(f"conv2       = {x.shape}")
        x = self.conv3(x)  # N x 96 x 62 x 400
        cprint(f"conv3       = {x.shape}")
        x = self.maxpool2(x)  # N x 96 x 31 x 200
        cprint(f"maxpool2    = {x.shape}")

        x = self.inception3a(x)
        cprint(f"inception3a = {x.shape}")
        x = self.inception3b(x)
        cprint(f"inception3b = {x.shape}")
        x = self.maxpool3(x)
        cprint(f"maxpool3    = {x.shape}")

        x = self.inception4a(x)
        cprint(f"inception4a = {x.shape}")
        x = self.inception4b(x)
        cprint(f"inception4b = {x.shape}")
        x = self.inception4c(x)
        cprint(f"inception4c = {x.shape}")
        x = self.inception4d(x)
        cprint(f"inception4d = {x.shape}")
        x = self.inception4e(x)
        cprint(f"inception4e = {x.shape}")

        x = self.maxpool4(x)
        cprint(f"maxpool4 = {x.shape}")

        x = self.inception5a(x)
        cprint(f"inception5a = {x.shape}")
        x = self.inception5b(x)
        cprint(f"inception5b = {x.shape}")

        x = self.avgpool(x)  # N x 512 x 1 x 1
        cprint(f"avgpool = {x.shape}")
        x = torch.flatten(x, 1)  # N x 512
        cprint(f"flatten = {x.shape}")
        x = self.dropout(x)
        cprint(f"dropout = {x.shape}")
        x = self.fc(x)  # N x num_classes
        cprint(f"fc = {x.shape}")
        global printEnabled
        printEnabled = False

        return x

    def forward(self, x: Tensor) -> Tensor:
        x = self._forward(x)
        return x

In [14]:
model = GoogLeNetLighter(init_weights=True, dropout=dropout).to(device)

In [15]:
# 6. Training Loop
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weightDecay)

counts = np.array([10170, 10245, 9225, 7935])
max_count = np.max(counts)
loss_weight = torch.from_numpy(max_count / counts).to(device).float()
criterion = nn.CrossEntropyLoss(weight=loss_weight)

In [16]:
patience = 15
counter = 0
best_val_acc = 0.0
best_model_state = None

print("Dropout: ",dropout)
print("weightDecay: ",weightDecay)
print("lr: ",lr)

for epoch in range(100):
    model.train()
    train_loss = 0
    correct_train = 0
    total_train = 0
    
    for batch in train_loader:
        X = batch[0].to(device).float()
        y = batch[1].to(device).long()
        
        optimizer.zero_grad()
        outputs = model(X)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train += y.size(0)
        correct_train += (predicted == y).sum().item()
        
    avg_train_loss = train_loss / len(train_loader)
    train_acc = (correct_train / total_train) * 100

    # ==========================
    # 2. VALIDATION PHASE
    # ==========================
    model.eval() # Turn off dropout for accurate testing
    val_loss = 0
    correct_val = 0
    total_val = 0
    
    with torch.no_grad(): # Don't calculate gradients for validation (saves memory)
        for batch in test_loader:
            X = batch[0].to(device).float()
            y = batch[1].to(device).long()
            
            outputs = model(X)
            loss = criterion(outputs, y)
            
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_val += y.size(0)
            correct_val += (predicted == y).sum().item()
            
    avg_val_loss = val_loss / len(test_loader)
    val_acc = (correct_val / total_val) * 100

    print(f"Epoch {epoch+1}: Train Loss={avg_train_loss:.4f} (Acc={train_acc:.2f}%) | Val Loss={avg_val_loss:.4f} (Acc={val_acc:.2f}%)")
     # --- EARLY STOPPING ---
    if (val_acc - best_val_acc) < 1.5:
        best_val_acc = val_acc
        best_model_state = copy.deepcopy(model.state_dict())
        torch.save(model.state_dict(), 'best_googlenet15_final.pth')
        print(f"  --> New Best! {best_val_acc:.2f}%")
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("  --> Early Stopping.")
            break

if best_model_state:
    model.load_state_dict(best_model_state)
    print(f"Finished. Best Acc: {best_val_acc:.2f}%")


Dropout:  0.5
weightDecay:  0.0001
lr:  0.0001
******************************
conv1       = torch.Size([32, 16, 62, 800])
maxpool1    = torch.Size([32, 16, 60, 796])
conv2       = torch.Size([32, 16, 60, 796])
conv3       = torch.Size([32, 48, 60, 796])
maxpool2    = torch.Size([32, 48, 30, 398])
inception3a = torch.Size([32, 128, 30, 398])
inception3b = torch.Size([32, 240, 30, 398])
maxpool3    = torch.Size([32, 240, 15, 199])
inception4a = torch.Size([32, 256, 15, 199])
inception4b = torch.Size([32, 256, 15, 199])
inception4c = torch.Size([32, 256, 15, 199])
inception4d = torch.Size([32, 264, 15, 199])
inception4e = torch.Size([32, 416, 15, 199])
maxpool4 = torch.Size([32, 416, 8, 99])
inception5a = torch.Size([32, 416, 8, 99])
inception5b = torch.Size([32, 512, 8, 99])
avgpool = torch.Size([32, 512, 1, 1])
flatten = torch.Size([32, 512])
dropout = torch.Size([32, 512])
fc = torch.Size([32, 4])
Epoch 1: Train Loss=1.2953 (Acc=36.60%) | Val Loss=1.4274 (Acc=27.63%)
Epoch 2: Train Los